In [41]:
import sklearn
import pandas as pd
import numpy as np

In [47]:
# Datensatz laden

daten = pd.read_json("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_HATE_GruppeDetail.json")
daten = daten.drop(axis=1, labels=["tag1", "tag2"])

print(daten)

           id                                               data  \
0     1112521  @user @user @user Weitaus schlimmer. Heute ist...   
1     1114995   Das Deutsche Kaiserreich soll wieder auferstehen   
2     1110545                   Die BRD ist eine einzige Schande   
3     1114326  @user @user Die Grünen....besser kann man das ...   
4     4112169  @user @user Scheiss deutsche Politiker! Mehr g...   
...       ...                                                ...   
3045  1223336  Unbequeme Wahrheit:   Sexuelle Belästigung ist...   
3046  1221963  Vor was habt ihr Angst Liebe Bürger !!!???? St...   
3047  2220834  @user Schön den Dummkopf #Oppermann von der Vo...   
3048  1220580  @user   Hoffentlich sind die Nationalisten dan...   
3049  2222357  @user Klar, was sollen Sie denn auch anderes s...   

                         label  
0                [KeineGruppe]  
1                [KeineGruppe]  
2                [KeineGruppe]  
3     [Politische Einstellung]  
4                [

In [ ]:
# Für die detaillierteren Annotationen bei Gruppe und Handlung:
# Labels aufspalten, so dass nur noch binäre Probleme vorliegen

In [61]:
# Annotationen konvertieren

ja = ["NEG", "VVH", "Gruppe", "Handlung", "Nationalität", 'ethnische Herkunft / "Rasse"', "Religion / Weltanschauung",
        "Politische Einstellung", "Geschlecht", "Anderes Merkmal", "Aufstachelung zu Hass", "Aufforderung zu Gewalt- oder Willkürmaßnahmen", "Angriff der Menschenwürde"]

nein = ["KEINE", "KeineGruppe", "KeineHandlung"]

labels_vvh = ["KEINE", "VVH"]
labels_gruppe = ["Gruppe", "KeineGruppe"]
labels_handlung = ["KeineHandlung", "Handlung"]

def transform_label(label):
    if label in ja: return 1
    else: return 0

targets = list(map(transform_label, list(daten["label"])))

In [62]:
# Stratifizierter Train/Test-Split
# s. https://scikit-learn.org/stable/modules/cross_validation.html#stratification

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(daten["data"], targets, test_size=0.33, random_state=36, stratify=targets)


KeyError: 'key of type tuple not found and not a MultiIndex'

In [32]:
# Preprocessing & Feature Extraction

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Features:
# 1. Zeichen-N-Gramme, char_wb: Unigramme innerhalb von Wortgrenzen (Leerzeichen)
vectorizer = CountVectorizer(analyzer='char_wb', ngram_range=(1, 4), lowercase=True)
X_train = vectorizer.fit_transform(X_train) #vectorizer.fit_transform(train_data[1])
X_train = X_train.toarray()
X_test = vectorizer.transform(X_test) #vectorizer.transform(test_data[1])
X_test = X_test.toarray()
# X_train, X_test, y_train, y_test = X_train, X_test, y_train, y_test


In [133]:
# 2. doc2vec (in der gensim-Implementierung)

# s. https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html


import gensim
#from gensim import corpora, models, similarities, downloader
from gensim.test.test_doc2vec import ConcatenatedDoc2Vec


def read_corpus(datei, tokens_only=False):
    with open(datei, encoding="utf-8") as f:
        for i, line in enumerate(f):
            id, tweet, anno = line.strip().split("\t")
            # TODO: ändern, so dass nicht alles lowercase ist
            tokens = gensim.utils.simple_preprocess(tweet)
            if tokens_only:
                yield tokens
            else:
                # For training data, add tags
                # Tag z.B. Zeilennummer ([i]), oder: Korpus-ID
                yield gensim.models.doc2vec.TaggedDocument(tokens, [id])

train_corpus = list(read_corpus("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Train_HATE_Handlung.txt"))
test_corpus = list(read_corpus("..\Korpora\Referenzdatensatz_HateSpeech_Deutsch\HateSpeechDe_Test_HATE_Handlung.txt"))


# Distributed Memory
model_dm = gensim.models.doc2vec.Doc2Vec(vector_size=300, min_count=2, epochs=30)
model_dm.build_vocab(train_corpus)
model_dm.train(train_corpus, total_examples=model_dm.corpus_count, epochs=model_dm.epochs)

# Distributed Bag of Words
model_dbow = gensim.models.doc2vec.Doc2Vec(dm=0, vector_size=300, min_count=2, epochs=30)
model_dbow.build_vocab(train_corpus)
model_dbow.train(train_corpus, total_examples=model_dbow.corpus_count, epochs=model_dbow.epochs)

# zur Kombination von DBOW und DM
concat_model = ConcatenatedDoc2Vec([model_dbow, model_dm])

# Modell speichern


# Einzelnen Vektor berechnen (für einen bereits vorverarbeiteten, tokenisierten String): 
#vector = concat_model.infer_vector(gensim.utils.simple_preprocess( "Die gr\u00f6\u00dfte Terrororganisation der Welt ist die USA und ihrer Ostk\u00fcste!"))
#print(vector)

# Alle Vektoren berechnen

# von https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4;
# regressors? steps?
def feature_vectors(model, docs, tags):
    targets, tags_final = zip(*[(model.infer_vector(doc.words, epochs=20), tags[line]) for line, doc in enumerate(docs)])
    return targets, tags_final


# Train/Test Split; durch die Dateien vorgegeben, ca 70/30
X_train, y_train  = feature_vectors(concat_model, train_corpus, tags_train)
X_test, y_test  = feature_vectors(concat_model, test_corpus, tags_test)

# Alternativ: mit der gensim-eigenen API für sklearn
# s. https://radimrehurek.com/gensim_3.8.3/sklearn_api/d2vmodel.html
# from gensim.sklearn_api import D2VTransformer


In [37]:
# Training
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression # war wohl ganz gut (Projekt FU)
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()
model.fit(X_train,y_train)

RandomForestClassifier()

In [38]:
# Evaluation

from sklearn import metrics
from sklearn.metrics import PrecisionRecallDisplay, precision_recall_curve


predicted = model.predict(X_test)
# np.mean(predicted==y_test) # Accuracy

#precision, recall, _ = precision_recall_curve(y_test, predicted, pos_label={'Gruppe', 'KeineGruppe'})
#disp = PrecisionRecallDisplay(precision=precision, recall=recall)
#disp.plot()

#fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted, pos_label={'Gruppe', 'KeineGruppe'})
#metrics.auc(fpr, tpr)

print(str(metrics.classification_report(y_test, predicted, target_names=labels_gruppe)))
print(str(metrics.confusion_matrix(y_test, predicted)))

#Bayes, Probeklassifikation
"""              precision    recall  f1-score   support

    VVH-Allg       0.40      0.19      0.26        21
       Keine       0.94      0.98      0.96       286

    accuracy                           0.93       307
   macro avg       0.67      0.58      0.61       307
weighted avg       0.91      0.93      0.91       307

[[  4  17]
 [  6 280]]

 precision    recall  f1-score   support

    VVH-Allg       0.00      0.00      0.00        23
       Keine       0.93      1.00      0.96       284

    accuracy                           0.93       307
   macro avg       0.46      0.50      0.48       307
weighted avg       0.86      0.93      0.89       307
"""


              precision    recall  f1-score   support

      Gruppe       0.82      0.61      0.70       413
 KeineGruppe       0.77      0.91      0.83       594

    accuracy                           0.78      1007
   macro avg       0.80      0.76      0.77      1007
weighted avg       0.79      0.78      0.78      1007

[[250 163]
 [ 54 540]]


'              precision    recall  f1-score   support\n\n    VVH-Allg       0.40      0.19      0.26        21\n       Keine       0.94      0.98      0.96       286\n\n    accuracy                           0.93       307\n   macro avg       0.67      0.58      0.61       307\nweighted avg       0.91      0.93      0.91       307\n\n[[  4  17]\n [  6 280]]\n\n precision    recall  f1-score   support\n\n    VVH-Allg       0.00      0.00      0.00        23\n       Keine       0.93      1.00      0.96       284\n\n    accuracy                           0.93       307\n   macro avg       0.46      0.50      0.48       307\nweighted avg       0.86      0.93      0.89       307\n'

In [197]:
# Alternative: Cross validation

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB


cvmodel = MultinomialNB()
scores = cross_val_score(cvmodel, X=X.toarray(), y=Y, cv=5, scoring='f1_macro')

NameError: name 'Y' is not defined

In [ ]:
print(scores)